## Pipeline Flavor Profile

In [115]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import csv
import numpy as np


ERROR! Session/line number was not unique in database. History logging moved to new session 214


In [116]:
'''
does convert a pandas dataframe into a matrix where 
keys = row name
values = col name
data = either 0 or 1 
'''
def dict_to_matrix(dictionary):
    unique_values = set(val for sublist in dictionary.values() for val in sublist)
    data_dict = {key: {val: 1 if val in values else 0 for val in unique_values} for key, values in dictionary.items()}
    df = pd.DataFrame(data_dict).T
    return(df)


In [117]:
'''
find overlapp between to lists with a threshold (=70)
'''
def overlapping_elements(list1, list2, threshold=70):
    overlap = []
    for elem1 in list1:
        for elem2 in list2:
            ratio = fuzz.ratio(elem1, elem2)
            if ratio >= threshold:
                overlap.append(elem1)
    return overlap

In [118]:
'''
Convert DataFrame to dictionary
'''
def dataframe_to_dict(df):
    result_dict = {}
    for index, row in df.iterrows():
        if row[0] in result_dict:
            result_dict[row[0]].append(row[1])
        else:
            result_dict[row[0]] = [row[1]]
    return result_dict     

In [119]:
'''
find match between dictionary entries and the rows of a dataframe
'''
def find_matching_entries(dictionary,df):
    threshold = 80
    found_match = []
    found_match_per_entry = {}
    for key, value in dictionary.items():
        all_matches = set()
        for index, row in df.iterrows():
            for keyword in value:
                if fuzz.ratio(keyword, str(index).lower()) >= threshold:
                    all_matches.add(index)
                    found_match.append(row)
        found_match_per_entry[key] = all_matches
    found_match = pd.DataFrame(found_match)
    found_match = found_match.drop_duplicates()
    return(found_match, found_match_per_entry)

In [120]:
'''
create a new matrix by finding matches and filtering right columns
'''
def create_matrix_from_dict(dictionary,df, index):
    matrix = pd.DataFrame(columns=list(df), index=index)
    final_dict = {}
    for key, value in dictionary.items():
        current_val = []
        for v in value:
            for index, row in df.iterrows():
                if index == v:
                    current_val.append(','.join(row.loc[row.eq(1)].index.tolist()))
                    # todo: maybe exception -> do change if two ones in a row
                    matrix.loc[key][row.loc[row.eq(1)].index.tolist()] = 1
        final_dict[key] = current_val
    matrix = matrix.fillna(0)
    matrix = matrix.loc[:, (matrix != 0).any(axis=0)]
    return(final_dict, matrix)
   

#### 1. MS Data ----- Massbank -----> Molecules

-> Chris

save which intensities belong to which molecule and give each the % 
2 approaches for one molecule with multiple OQs
1.  each gets same % 
2.  divide % by count of OQs per molecule

<span style="color:lightblue">mol_int</span> : dict where key is the molecule name and value is the intensites from the mass spec data

In [121]:
# DUMMY DATA
intensities = [80, 40, 5]
ms_cas =['(E)-3-hexenol', "coumarin",'citral']
mol_int = {'(E)-3-hexenol': 80, "coumarin": 40 ,'citral': 5}

In [122]:
# actual ms data
file_path = 'data/cas_intensities.csv'
data = pd.read_csv(file_path, delimiter=',')
# only use first repliate
data = data[data["replicate"]==1]

In [123]:
sample_key = "000920"

In [124]:
used_data = data[data["sample_key"]==sample_key]
#filter columns that are 0
used_data = used_data.loc[:, (used_data != 0).any(axis=0)]
used_data

,sample_key,replicate,6485-40-1,3033-23-6,21284-22-0,38427-78-0,495-61-4,29873-99-2,6909-30-4,99-49-0,...,13466-78-9,483-76-1,36564-42-8,705-86-2,2825-91-4,713-95-1,20307-84-0,120021-96-7,586-62-9,710-04-3
0,000920,1,1.943542e+08,1.240854e+09,1.048295e+08,9.087319e+07,1.831223e+08,1.831223e+08,6.043091e+08,1.943542e+08,...,2.528854e+08,1.831223e+08,1.048295e+08,4.645706e+08,4.645706e+08,1.893777e+08,1.831223e+08,1.831223e+08,2.528854e+08,2.662124e+08


In [125]:
'''# find rows with biggest variances
df = used_data
row_combinations = list(itertools.combinations(df.index, 2))
distances = [abs(df.loc[i[0]] - df.loc[i[1]]).sum() for i in row_combinations]
max_distance_indices = row_combinations[distances.index(max(distances))]
print(df.loc[max_distance_indices[0]])
print(df.loc[max_distance_indices[1]])
#=> sample: 000879 and 000373'''

'# find rows with biggest variances\ndf = used_data\nrow_combinations = list(itertools.combinations(df.index, 2))\ndistances = [abs(df.loc[i[0]] - df.loc[i[1]]).sum() for i in row_combinations]\nmax_distance_indices = row_combinations[distances.index(max(distances))]\nprint(df.loc[max_distance_indices[0]])\nprint(df.loc[max_distance_indices[1]])\n#=> sample: 000879 and 000373'

In [126]:
used_data = used_data.drop(columns=['sample_key', 'replicate'])
#largest_entries = used_data.iloc[0].nlargest(100)
largest_entries = np.log1p(used_data.iloc[0])

# Create a new DataFrame with only the largest entries
data_top50 = pd.DataFrame({'Intensities': largest_entries})
data_top50

,Intensities
6485-40-1,19.085193
3033-23-6,20.939066
21284-22-0,18.467846
38427-78-0,18.324976
495-61-4,19.025665
...,...
713-95-1,19.059254
20307-84-0,19.025665
120021-96-7,19.025665
586-62-9,19.348447


In [127]:
ms_cas = list(data_top50.index)

In [128]:
intensities = list(data_top50['Intensities'])

<span style="color:#FBB714">I_vector</span> : dataframe with *molecules x intensities*

In [129]:
#I_vector = pd.DataFrame(data = mol_int, index = ["intensities"]).T
I_vector = pd.DataFrame(data= intensities,index = ms_cas, columns=["intensities"])
I_vector

,intensities
6485-40-1,19.085193
3033-23-6,20.939066
21284-22-0,18.467846
38427-78-0,18.324976
495-61-4,19.025665
...,...
713-95-1,19.059254
20307-84-0,19.025665
120021-96-7,19.025665
586-62-9,19.348447


#### 2. Molecules ----- Flavornet -----> OD 
Find ODs by matching molecules with databases using Flavornet

<span style="color:lightblue">CAS_to_odorants.csv</span> : table with following columns - molecule, CAS, Mol wt, odorant


##### table manipualtion done in R

```r
library(rlang)
library(tidyverse)

CAS_to_odorants <- read.table("~/Bioinformatik_20.21/Bachelorarbeit/CAS_to_mol.txt", sep = "\t", header = FALSE)
OD_to_mol <- read.table("~/Bioinformatik_20.21/Bachelorarbeit/OD_mol.txt", sep = "\t", header = FALSE)

OD_to_mol_sep <- OD_to_mol %>%
  separate_rows(V2, sep = ";\\s*") %>%
  mutate(V2 = trimws(V2))
OD_to_mol_final <- OD_to_mol_sep[, c("V2","V1")]
colnames(OD_to_mol_final) <- c("molecule", "OD")

CAS_to_odorants_sep <- CAS_to_odorants %>%
  separate_rows(V3, sep = ";\\s*") %>%
  mutate(V3 = trimws(V3))
colnames(CAS_to_odorants_sep) <- c("CAS","Mol Wt", "molecule")


merged_df <- merge(CAS_to_odorants_sep, OD_to_mol_final, by = "molecule")
write.csv(merged_df, file = "~/Bioinformatik_20.21/Bachelorarbeit/CAS_mol_OD.csv", row.names = FALSE)
```

*Flavornet only*

<span style="color:lightblue">mol_to_OD</span> : dict where key is the molecule and the descriptors for these molecules are the values

In [130]:
'''
read flavornet data and save it in mol_to_OD
'''
# only using flavornet data
mol_to_OD = {}
molecules = []
descriptors = []
CAS_numbers= []
with open('./data/CAS_mol_OD.csv', 'r',encoding='utf-8') as tabfile:
    reader = csv.reader(tabfile, delimiter=',')
    for row in reader: 
        molecule = row[0]
        CAS = row[1]
        OD = row[3]
        molecules.append(molecule)
        CAS_numbers.append(CAS)
        descriptors.append(OD)
        if CAS in mol_to_OD:
            mol_to_OD[CAS].add(OD)
        else:
            mol_to_OD[CAS]  = set([OD])


CAS_numbers = list(set(CAS_numbers))
#list with all molecules from flavornet
molecules = list(set(molecules))
# list of all individual descriptors/ODs
descriptors = list(set(descriptors))

<span style="color:#FBB714">D</span>: Dataframe with all *molecules x ODs* - rigth now only flavornet database

In [131]:
D = pd.DataFrame(index=CAS_numbers, columns=descriptors)

In [132]:
len(CAS_numbers)

732

In [133]:
for col in D.columns:
    for index, row in D.iterrows():
        # Check if value in  current row matches any entry in the dictionary
        if index in mol_to_OD.keys() and col in mol_to_OD[index]:
            # Set to 1
            D.at[index, col] = 1

D = D.fillna(0)
D

,pear,butter,baked,urine,paint,sweet,cotton candy,hummus,crushed bug,pea,...,potato,marshmallow,melon,cologne,cooked meat,caramel,tart lime,hazelnut,grass,plastic
122-00-9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105-54-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28588-75-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120-72-9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104-61-0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118-61-6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143-07-7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24415-26-7,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105-66-8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


match molecules from MS data with flavors

In [134]:
flavors = []
mol_flavors = {}
for mol in ms_cas:
    if mol in mol_to_OD.keys():
        flavors += mol_to_OD[mol]
        mol_flavors[mol] = mol_to_OD[mol]
    else:
        mol_flavors[mol] = {}

found_flavors = set(flavors)



<span style="color:lightblue">mol_flavors</span>: contains the molecule from where each OD is coming from 

In [135]:
mol_flavors

{'6485-40-1': {'mint'},
 '3033-23-6': {'rose', 'sweet'},
 '21284-22-0': {'green tea', 'herb', 'spice'},
 '38427-78-0': {'terpentine'},
 '495-61-4': {'balsamic'},
 '29873-99-2': {'green', 'oil', 'wood'},
 '6909-30-4': {'green'},
 '99-49-0': {'basil', 'caraway', 'fennel', 'mint'},
 '2244-16-8': {'caraway'},
 '876-17-5': {'flower', 'green'},
 '5989-27-5': {'citrus', 'mint'},
 '4674-50-4': {'grapefruit'},
 '2243-33-6': {'dill'},
 '10307-61-6': {'apple'},
 '69064-37-5': {'oil'},
 '13419-69-7': {'fat', 'must'},
 '928-95-0': {'green', 'leaf', 'walnut'},
 '18829-56-6': {'cucumber', 'fat', 'green'},
 '53448-07-0': {'fat', 'green', 'soap'},
 '928-97-2': {'fresh', 'moss'},
 '1197-07-5': {'caraway', 'solvent'},
 '14371-10-9': {'cinnamon', 'paint'},
 '09.04.5948': {'herb', 'warm'},
 '106-28-5': {'muguet'},
 '5273-85-8': {'flower', 'spice'},
 '5932-68-3': {'flower'},
 '4959-35-7': {'citrus', 'fresh'},
 '34995-77-2': {'flower'},
 '39638-67-0': {'coconut', 'flower'},
 '7212-40-0': {'fresh', 'mint'},
 

<span style="color:lightblue">found_flavors</span>: list of the ODs found from the identified molecules 


In [136]:
found_flavors

{'acid',
 'alkane',
 'almond',
 'almond shell',
 'amine',
 'anise',
 'apple',
 'apple peel',
 'apple. rose',
 'apricot',
 'baked',
 'balsamic',
 'banana',
 'basil',
 'beet',
 'biscuit',
 'bitter almond',
 'black currant',
 'boiled vegetable',
 'box tree',
 'bread',
 'broccoli',
 'brown sugar',
 'burnt',
 'burnt sugar',
 'butter',
 'butterscotch',
 'cabbage',
 'camomile',
 'camphor',
 'caramel',
 'caraway',
 'cardboard',
 'carrot',
 'cat',
 'celery',
 'cheese',
 'chemical',
 'cinnamon',
 'citrus',
 'clove',
 'cocoa',
 'coconut',
 'coffee',
 'cognac',
 'cologne',
 'cooked meat',
 'cooked potato',
 'coriander',
 'cotton candy',
 'coumarin',
 'cream',
 'crushed bug',
 'cucumber',
 'curry',
 'dill',
 'dust',
 'earth',
 'ester',
 'ether',
 'fat',
 'fecal',
 'fennel',
 'fish',
 'flower',
 'foxy',
 'fresh',
 'fried',
 'fruit',
 'garlic',
 'gasoline',
 'geranium',
 'grape',
 'grapefruit',
 'grass',
 'green',
 'green bean',
 'green leaf',
 'green pepper',
 'green tea',
 'hawthorne',
 'hazelnut',

<span style="color:#FBB714">OD</span>: Dataframe - *molecules x ODs*

In [137]:
# filters only relevant rows - only those ODs coming from indentified molecules
OD = D[(D.index).isin(I_vector.index)]
# get only important/matching columns
OD = OD.loc[:, (OD != 0).any(axis=0)]
OD 

,pear,butter,baked,urine,paint,sweet,cotton candy,hummus,crushed bug,pea,...,potato,marshmallow,melon,cologne,cooked meat,caramel,tart lime,hazelnut,grass,plastic
122-00-9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105-54-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28588-75-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120-72-9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104-61-0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118-61-6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143-07-7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24415-26-7,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105-66-8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 3. OD ----------> OQ


### Whisky Research institute

In [138]:
Research_data = pd.read_csv('./data/swri_flavorwheel.csv', sep=';')
Research_OQ_OD =  Research_data.drop(columns=Research_data.columns[0])


In [139]:
OD_OQ = dict_to_matrix(dataframe_to_dict(Research_OQ_OD)).T

<span style="color:#FBB714">OD_OQ</span>: Dataframe - all ODs x OQs from research institute

In [140]:
OD_OQ

,Burnt,Smoky,Medicinal,Leathery,Tobacco,Sweaty,Dry cereals,Wet cereals,leafy,Herbal,...,Solvently,Soapy,Sweet,Woody,Nutty,Spicy,Oily,Sour,Sulphury,Stale
TCP,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
leather upholstery,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Burnt toast,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Green/grassy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Raisins,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tyres,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
spent fireworks,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Geranium leaves,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Horse saddle,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
# find matching OQs to found ODs from massspec x flavornet
matching_OQs, matching_OQs_dict = find_matching_entries(mol_flavors, OD_OQ)

In [142]:
# get only important columns
matching_OQs = matching_OQs.loc[:, (matching_OQs != 0).any(axis=0)]
matching_OQs

,Burnt,Sweaty,Dry cereals,Green vegetables,Fresh flowers,Fresh fruit,Citrus,Artificial fruit flavourings,Vanilla,Toffee,...,Cooked vegetables,Struck match,Earthy,Fruity,Solvently,Soapy,Sweet,Woody,Spicy,Oily
Roses,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sweet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Spicy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Woody,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Oily,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
grapefruit,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Apples,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cucumber,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Soapy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Solvently,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [143]:
# to know which OD comes from which molecule
#remove empty entries
matching_OQs_dict = {key: value for key, value in matching_OQs_dict.items() if value}
matching_OQs_dict

{'3033-23-6': {'Roses', 'Sweet'},
 '21284-22-0': {'Spicy'},
 '29873-99-2': {'Oily', 'Woody'},
 '4674-50-4': {'grapefruit'},
 '10307-61-6': {'Apples'},
 '69064-37-5': {'Oily'},
 '18829-56-6': {'cucumber'},
 '53448-07-0': {'Soapy'},
 '1197-07-5': {'Solvently'},
 '14371-10-9': {'cinnamon'},
 '5273-85-8': {'Spicy'},
 '13474-59-4': {'Woody'},
 '23726-91-2': {'Apples'},
 '3779-61-1': {'Sweet'},
 '4128-17-0': {'Oily'},
 '928-96-1': {'Gassy'},
 '53398-85-9': {'Sweet'},
 '31501-11-8': {'Fruity '},
 '15456-69-6': {'Soapy'},
 '18679-18-0': {'Sweet'},
 '55013-32-6': {'Spicy'},
 '562-74-3': {'nutmeg'},
 '1197-15-5': {'Spicy'},
 '470-67-7': {'Spicy'},
 '470-82-6': {'Sweet'},
 '15051-81-7': {'Sweet', 'Woody'},
 '75853-49-5': {'Sweaty', 'Sweet'},
 '2497-25-8': {'oranges'},
 '3913-71-1': {'oranges'},
 '3913-81-3': {'oranges'},
 '20407-84-5': {'Sweet'},
 '104-76-7': {'Roses'},
 '100-71-0': {'Gassy'},
 '928-94-9': {'Fruity '},
 '624-41-9': {'Fruity '},
 '628-99-9': {'cucumber'},
 '2463-53-8': {'cucumber'

<span style="color:#FBB714">OQs</span>: Dataframe - ODs x OQs

In [144]:
OQ_dict, OQs = create_matrix_from_dict(matching_OQs_dict, matching_OQs,matching_OQs_dict.keys())

In [145]:
OQs

,Burnt,Sweaty,Dry cereals,Green vegetables,Fresh flowers,Fresh fruit,Citrus,Artificial fruit flavourings,Vanilla,Toffee,...,Cooked vegetables,Struck match,Earthy,Fruity,Solvently,Soapy,Sweet,Woody,Spicy,Oily
3033-23-6,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
21284-22-0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
29873-99-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4674-50-4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10307-61-6,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13466-78-9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
483-76-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
713-95-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
20307-84-0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [146]:
OQ_dict

{'3033-23-6': ['Fresh flowers', 'Sweet'],
 '21284-22-0': ['Spicy'],
 '29873-99-2': ['Woody', 'Oily'],
 '4674-50-4': ['Citrus'],
 '10307-61-6': ['Fresh fruit'],
 '69064-37-5': ['Oily'],
 '18829-56-6': ['Green vegetables'],
 '53448-07-0': ['Soapy'],
 '1197-07-5': ['Solvently'],
 '14371-10-9': ['Dried spice'],
 '5273-85-8': ['Spicy'],
 '13474-59-4': ['Woody'],
 '23726-91-2': ['Fresh fruit'],
 '3779-61-1': ['Sweet'],
 '4128-17-0': ['Oily'],
 '928-96-1': ['Struck match '],
 '53398-85-9': ['Sweet'],
 '31501-11-8': ['Fruity '],
 '15456-69-6': ['Soapy'],
 '18679-18-0': ['Sweet'],
 '55013-32-6': ['Spicy'],
 '562-74-3': [],
 '1197-15-5': ['Spicy'],
 '470-67-7': ['Spicy'],
 '470-82-6': ['Sweet'],
 '15051-81-7': ['Woody', 'Sweet'],
 '75853-49-5': ['Sweaty', 'Sweet'],
 '2497-25-8': [],
 '3913-71-1': [],
 '3913-81-3': [],
 '20407-84-5': ['Sweet'],
 '104-76-7': ['Fresh flowers'],
 '100-71-0': ['Struck match '],
 '928-94-9': ['Fruity '],
 '624-41-9': ['Fruity '],
 '628-99-9': ['Green vegetables'],
 '2

#### 4.  OQ -----------> OSA


<span style="color:#FBB714">OQ_OSA</span>: Dataframe - all OQs x OSAs from research institute

In [147]:
Research_OSA_OQ = Research_data.drop(columns=Research_data.columns[2])
OQ_OSA = dict_to_matrix(dataframe_to_dict(Research_OSA_OQ)).T

In [148]:
OQ_OSA

,peaty,feinty,cereal,Green/grassy,Floral,Fruity,Solvently,Soapy,Sweet,Woody,Nutty,Spicy,Oily,Sour,Sulphury,Stale
Floral,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
Peppery,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
feinty,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Burnt,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Citrus,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Soapy,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Tobacco,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Sweet,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
Green/grassy,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Honey,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


 #### 4.1 Binary Matrix

In [149]:
# find matching OQs to found ODs from massspec x flavornet
matching_OSAs, matching_OSAs_dict = find_matching_entries(OQ_dict, OQ_OSA)

In [150]:
print(matching_OSAs)

                  peaty  feinty  cereal  Green/grassy  Floral  Fruity   \
Sweet                 0       0       0             0       0        0   
Fresh flowers         0       0       0             0       1        0   
Spicy                 0       0       0             0       0        0   
Woody                 0       0       0             0       0        0   
Citrus                0       0       0             0       0        1   
Green vegetables      0       0       0             1       0        0   
Soapy                 0       0       0             0       0        0   
Solvently             0       0       0             0       0        0   
Struck match          0       0       0             0       0        0   
Sweaty                0       1       0             0       0        0   
Earthy                0       0       0             0       0        0   
Buttery               0       0       0             0       0        0   
Toasted               0       0       

In [151]:
print(matching_OSAs.loc[:, (matching_OSAs != 0).any(axis=0)])

                  peaty  feinty  cereal  Green/grassy  Floral  Fruity   \
Sweet                 0       0       0             0       0        0   
Fresh flowers         0       0       0             0       1        0   
Spicy                 0       0       0             0       0        0   
Woody                 0       0       0             0       0        0   
Citrus                0       0       0             0       0        1   
Green vegetables      0       0       0             1       0        0   
Soapy                 0       0       0             0       0        0   
Solvently             0       0       0             0       0        0   
Struck match          0       0       0             0       0        0   
Sweaty                0       1       0             0       0        0   
Earthy                0       0       0             0       0        0   
Buttery               0       0       0             0       0        0   
Toasted               0       0       

<span style="color:#FBB714">OSAs</span>: Dataframe - OQs x OSAs

In [152]:
OSA_dict, OSAs = create_matrix_from_dict(matching_OSAs_dict, matching_OSAs, matching_OSAs_dict.keys())

In [153]:
print(OSAs)

             peaty  feinty  cereal  Green/grassy  Floral  Fruity   Solvently  \
3033-23-6        0       0       0             0       1        0          0   
21284-22-0       0       0       0             0       0        0          0   
29873-99-2       0       0       0             0       0        0          0   
4674-50-4        0       0       0             0       0        1          0   
10307-61-6       0       0       0             0       0        0          0   
...            ...     ...     ...           ...     ...      ...        ...   
13466-78-9       0       0       0             0       0        0          0   
483-76-1         0       0       0             0       0        0          0   
713-95-1         0       0       0             0       0        0          0   
20307-84-0       0       0       0             0       0        0          0   
120021-96-7      0       0       0             0       0        0          0   

             Soapy  Sweet  Woody  Nutty

In [154]:
# Cas numbers and intensities for which we found OQs 
final_I_vector = I_vector[I_vector.index.isin(OSA_dict.keys())]

In [155]:
final_I_vector

,intensities
3033-23-6,20.939066
21284-22-0,18.467846
29873-99-2,19.025665
4674-50-4,18.324976
10307-61-6,18.236496
...,...
13466-78-9,19.348447
483-76-1,19.025665
713-95-1,19.059254
20307-84-0,19.025665


In [156]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
I_vector_normalized = normalizer.fit_transform(final_I_vector.T).T
I_vector_normalized

array([[0.06085556],
       [0.05367341],
       [0.05529461],
       [0.05325818],
       [0.05300104],
       [0.05638198],
       [0.05485999],
       [0.05594842],
       [0.05876455],
       [0.05356996],
       [0.05611333],
       [0.05529461],
       [0.05545065],
       [0.05623272],
       [0.05149549],
       [0.05335527],
       [0.05638198],
       [0.05539223],
       [0.05444059],
       [0.05444059],
       [0.05671438],
       [0.06085556],
       [0.05546762],
       [0.06085556],
       [0.06085556],
       [0.05367341],
       [0.0549169 ],
       [0.06085556],
       [0.06085556],
       [0.06085556],
       [0.05958408],
       [0.05300104],
       [0.05166745],
       [0.05335527],
       [0.05300104],
       [0.05805364],
       [0.05485999],
       [0.05485999],
       [0.05300104],
       [0.06085556],
       [0.05588857],
       [0.05594842],
       [0.05876455],
       [0.05335527],
       [0.05528016],
       [0.05751639],
       [0.05671438],
       [0.056

In [157]:
final_OSA = OSAs*I_vector_normalized
final_OSA

,peaty,feinty,cereal,Green/grassy,Floral,Fruity,Solvently,Soapy,Sweet,Woody,Nutty,Spicy,Oily,Sour,Sulphury,Stale
3033-23-6,0.0,0.0,0.0,0.0,0.060856,0.000000,0.0,0.0,0.060856,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
21284-22-0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.053673,0.0,0.0,0.0,0.0
29873-99-2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.055295,0.0,0.000000,0.0,0.0,0.0,0.0
4674-50-4,0.0,0.0,0.0,0.0,0.000000,0.053258,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
10307-61-6,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13466-78-9,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
483-76-1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.055295,0.0,0.000000,0.0,0.0,0.0,0.0
713-95-1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.055392,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
20307-84-0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.055295,0.0,0.000000,0.0,0.0,0.0,0.0


In [197]:
final_OSA.sum(axis=0)*10

peaty            0.551070
feinty           4.825058
cereal           2.786105
Green/grassy     6.807675
Floral           8.974375
Fruity           1.112585
Solvently        1.666697
Soapy            6.670317
Sweet           36.902148
Woody           25.442252
Nutty            2.686850
Spicy           23.962436
Oily             1.616430
Sour             3.228415
Sulphury         2.150924
Stale            7.628535
dtype: float64

#### 4.2 Fuzzy Logic 
Use Fuzzy logic to weight OQ with expert knowledge/Intensity from Masspec


In [158]:
# list with OSA and corresponding OQs -> look at inner circle of flavorwheel
OSAs_corresponding_OQs = dataframe_to_dict(Research_OSA_OQ.drop_duplicates())
OSAs_corresponding_OQs

{'peaty': ['Burnt', 'Smoky', 'Medicinal', 'peaty'],
 'feinty': ['Leathery', 'Tobacco', 'Sweaty', 'feinty'],
 'cereal': ['Dry cereals', 'Wet cereals', 'cereal'],
 'Green/grassy': ['leafy', 'Herbal', 'Green vegetables', 'Green/grassy'],
 'Floral': ['Fresh flowers', 'Perfumed', 'Floral'],
 'Fruity ': ['Fresh fruit',
  'Dried fruit',
  'Citrus',
  'Artificial fruit flavourings',
  'Fruity '],
 'Solvently': ['Nail varnish remover', 'Solvently'],
 'Soapy': ['Unperfumed soap', 'Soapy'],
 'Sweet': ['Vanilla', 'Honey', 'Toffee', 'Sweet'],
 'Woody': ['New wood', 'Okay', 'Woody'],
 'Nutty': ['Coconut', 'Nuts', 'Toasted', 'Nutty'],
 'Spicy': ['Dried spice', 'Peppery', 'Spicy'],
 'Oily': ['Buttery ', 'Waxy', 'Oils/fats', 'Oily'],
 'Sour': ['Vinegary', 'Cheesy', 'Vomit', 'Sour'],
 'Sulphury': ['Cooked vegetables ',
  'Rubbery',
  'Struck match ',
  'Decaying',
  'Meaty ',
  'Sulphury'],
 'Stale': ['Earthy', 'Musty', 'Metallic', 'Stale']}

In [159]:
# all found OQs for our mass spec data
found_OQs =set(item for sublist in OQ_dict.values() for item in sublist)
found_OQs

{'Artificial fruit flavourings',
 'Burnt',
 'Buttery ',
 'Citrus',
 'Cooked vegetables ',
 'Dried spice',
 'Dry cereals',
 'Earthy',
 'Fresh flowers',
 'Fresh fruit',
 'Fruity ',
 'Green vegetables',
 'Nuts',
 'Oily',
 'Soapy',
 'Solvently',
 'Spicy',
 'Struck match ',
 'Sweaty',
 'Sweet',
 'Toasted',
 'Toffee',
 'Vanilla',
 'Vinegary',
 'Vomit',
 'Woody'}

1. normalize OQs

In [160]:
CAS_intensities_OQs  = final_I_vector.copy()
CAS_intensities_OQs["OQs"]= OQ_dict
CAS_intensities_OQs

,intensities,OQs
3033-23-6,20.939066,"[Fresh flowers, Sweet]"
21284-22-0,18.467846,[Spicy]
29873-99-2,19.025665,"[Woody, Oily]"
4674-50-4,18.324976,[Citrus]
10307-61-6,18.236496,[Fresh fruit]
...,...,...
13466-78-9,19.348447,[]
483-76-1,19.025665,[Woody]
713-95-1,19.059254,"[Sweet, Fruity ]"
20307-84-0,19.025665,[Woody]


add up all intensities per OQ 

intensity_per_OQ: dict that contains the OQ and the summed up intensities

In [161]:
intensity_per_OQ = {}
for found_OQ in found_OQs:
    for index, row in CAS_intensities_OQs.iterrows():
        if found_OQ in row["OQs"]:
            if found_OQ in intensity_per_OQ:
                intensity_per_OQ[found_OQ] += row["intensities"]
            else:
                intensity_per_OQ[found_OQ] = row["intensities"]

In [162]:
'''
normalizes the OQs by dividing each row/cell by the overall intensities, meaning the sum of all intensities
'''
def normalize_OQs(df_intensities):
    normalized_df = pd.DataFrame(columns=["normalized_intensities"])
    overall_intensities = df_intensities["intensities"].sum()
    normalized_df["normalized_intensities"] = df_intensities/overall_intensities
    return(normalized_df)

In [163]:
OQ_intensities = pd.DataFrame(intensity_per_OQ.values(), index = intensity_per_OQ.keys(), columns=["intensities"])

In [164]:
OQ_intensities

,intensities
Citrus,38.281600
Soapy,229.510996
Burnt,18.961113
Sweet,1269.722134
Spicy,824.494967
Nuts,74.876340
Dry cereals,95.863775
Vanilla,18.500567
Sweaty,166.019695
Vomit,111.082688


In [165]:
normalized_OQ_intensities = normalize_OQs(OQ_intensities)
normalized_OQ_intensities

,normalized_intensities
Citrus,0.005516
Soapy,0.033070
Burnt,0.002732
Sweet,0.182952
Spicy,0.118800
Nuts,0.010789
Dry cereals,0.013813
Vanilla,0.002666
Sweaty,0.023921
Vomit,0.016006


2. Looking at each OSA indiviudally to find intensity

In [166]:
'''
find all OQs that are corresponding to one specific OSA
'''
def find_OQ_per_OSA_with_intensities(OSA):
    OQ_per_OSA = {}
    for consisting_OQs in OSAs_corresponding_OQs[OSA]:
        if consisting_OQs in normalized_OQ_intensities.index:
            OQ_per_OSA[consisting_OQs] = normalized_OQ_intensities.loc[consisting_OQs]
    # normalize so that all intensities per OSA sums up to 1
    df_OQ_per_OSA = pd.DataFrame(OQ_per_OSA).T
    df_OQ_per_OSA = df_OQ_per_OSA.div(df_OQ_per_OSA['normalized_intensities'].sum())
    return(df_OQ_per_OSA)

equation_2:
The function µ(𝑥) is defined as:

$$
\mu(𝑥) =
\begin{cases}
0 & \text{if } 𝑥 ≤ 𝑎_1 \\
\frac{𝑥−𝑎_1}{𝑎_2−𝑎_1} & \text{if } 𝑎_1 < 𝑥 ≤ 𝑎_2 \\
1 & \text{if } 𝑎_2 < 𝑥 ≤ 𝑎_3 \\
\frac{𝑎_4−𝑥}{𝑎_4−𝑎_3} & \text{if } 𝑎_3 < 𝑥 ≤ 𝑎_4 \\
0 & \text{if } 𝑎_4 < 𝑥 \\
\end{cases}
$$


In [167]:
#most of the time -> TODO: optimize these with cross validation  
a1 = 0
a2 = 0
a3 = 0.33
a4 = 1

In [168]:
def equation_2(x,a1,a2,a3,a4):
    if x <= a1:
        µ = 0
    elif a1 < x and x <= a2:
        µ = (x-a1)/(a2-a1)
    elif a2 < x and x <= a3:
        µ = 1
    elif a3 < x and x <= a4:
        µ = (a4-x)/(a4-a3)
    elif a4 < x:
        µ = 0
    return µ

equation3: 

𝐼𝑛𝑡𝑒𝑛𝑠𝑖𝑡𝑦(𝑂𝑆𝐴) = 10 × ∏^i 1 µ(𝑂𝑄𝑖)

In [169]:
def equation_3(µ_OQs_list):
    intensity_OSA = 10 * np.prod(µ_OQs_list)
    return(intensity_OSA)


e.g. with OSA sweet

In [170]:
# e.g sweet consists of: OQs ['Vanilla', 'Honey', 'Toffee', 'Sweet']
OSAs_corresponding_OQs['Sweet']

['Vanilla', 'Honey', 'Toffee', 'Sweet']

In [171]:
# sweet 
sweet = find_OQ_per_OSA_with_intensities('Sweet')

In [172]:
sweet

,normalized_intensities
Vanilla,0.013759
Toffee,0.041939
Sweet,0.944302


In [173]:
OQ1 = equation_2(sweet.loc["Sweet"]["normalized_intensities"],a1,a2,0.33,a4)
OQ2= equation_2(sweet.loc["Vanilla"]["normalized_intensities"],a1,a2,0.033,a4)
OQ3= equation_2(sweet.loc["Toffee"]["normalized_intensities"],a1,a2,0.043,a4)
equation_3([OQ1,OQ2,OQ3]) 

0.8313135149939276

for all OSAs

In [174]:
all_OSAs = list(OSAs_corresponding_OQs.keys())

In [175]:
OSAs_int = {}
for OSA in all_OSAs: 
   OSAs_int[OSA] = 0
   current_OSA = find_OQ_per_OSA_with_intensities(OSA)
   OQ_int = []
   for OQ in current_OSA.index:
      OQ_int.append(equation_2(current_OSA.loc[OQ]["normalized_intensities"],a1,a2,a3,a4))
   OSAs_int[OSA] = equation_3(OQ_int)

In [176]:
OSAs_int = {}
for OSA in all_OSAs: 
   OSAs_int[OSA] = 0
   current_OSA = find_OQ_per_OSA_with_intensities(OSA)
   OQ_int = []
   for OQ in current_OSA.index:
      OQ_int.append(current_OSA.loc[OQ]["normalized_intensities"])
   OSAs_int[OSA] = equation_3(OQ_int)

In [177]:
OSAs_int

{'peaty': 10.0,
 'feinty': 10.0,
 'cereal': 10.0,
 'Green/grassy': 10.0,
 'Floral': 10.0,
 'Fruity ': 0.00042240598303466966,
 'Solvently': 10.0,
 'Soapy': 10.0,
 'Sweet': 0.00544899195662637,
 'Woody': 10.0,
 'Nutty': 2.4724275232956225,
 'Spicy': 0.5953939676957896,
 'Oily': 1.1571227024779875,
 'Sour': 1.8573470550000724,
 'Sulphury': 2.406743226319665,
 'Stale': 10.0}

In [178]:
df_OSAs_int = pd.DataFrame(OSAs_int.values(), index = OSAs_int.keys(), columns = ["Intensities"])

In [179]:
df_OSAs_int

,Intensities
peaty,10.000000
feinty,10.000000
cereal,10.000000
Green/grassy,10.000000
Floral,10.000000
Fruity,0.000422
Solvently,10.000000
Soapy,10.000000
Sweet,0.005449
Woody,10.000000


In [180]:
plot_fuzzy_OSA = df_OSAs_int[df_OSAs_int['Intensities'] != 0]
plot_fuzzy_OSA

,Intensities
peaty,10.000000
feinty,10.000000
cereal,10.000000
Green/grassy,10.000000
Floral,10.000000
Fruity,0.000422
Solvently,10.000000
Soapy,10.000000
Sweet,0.005449
Woody,10.000000


In [181]:
plot_fuzzy_OSA.index

Index(['peaty', 'feinty', 'cereal', 'Green/grassy', 'Floral', 'Fruity ',
       'Solvently', 'Soapy', 'Sweet', 'Woody', 'Nutty', 'Spicy', 'Oily',
       'Sour', 'Sulphury', 'Stale'],
      dtype='object')

40,30,50,40,30,40,30,40,60
40,30,50,30,40,30,50,20,40
40,30,50,30,40,30,50,20,40
floral,fruit,wood,complexity,malt,sweet,peat,dried_fruits,aftertaste,date

[3,4,0,0,0,0,0]

optimization with cross validation LOO

#### 4.3 Comparison BM and FL

In [182]:
np.array(final_OSA.sum()*10)

array([ 0.55107   ,  4.82505829,  2.786105  ,  6.80767496,  8.97437513,
        1.11258457,  1.66669663,  6.67031665, 36.90214783, 25.44225166,
        2.68685043, 23.96243582,  1.61643025,  3.22841483,  2.15092366,
        7.62853493])

In [183]:
final_OSA.columns

Index(['peaty', 'feinty', 'cereal', 'Green/grassy', 'Floral', 'Fruity ',
       'Solvently', 'Soapy', 'Sweet', 'Woody', 'Nutty', 'Spicy', 'Oily',
       'Sour', 'Sulphury', 'Stale'],
      dtype='object')

In [184]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
  r=(plot_fuzzy_OSA["Intensities"]),
  theta=list(plot_fuzzy_OSA.index),
  fill="toself",
  name='ms fuzzy'
))
fig.add_trace(go.Scatterpolar(
  r=([ 0.7776613, 23.58432344,1.00636248, 16.65879529, 0.50338256,  1.16029695,  0.75458068]),
  theta=list(plot_fuzzy_OSA.index),
  fill="toself",
  name='ms binary'
))
fig.add_trace(go.Scatterpolar(
  r=([3,4,0,0,0,0,0]),
  theta=list(plot_fuzzy_OSA.index),
  fill="toself",
  name='panelist 1'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
       range=[0, 10]
    ),
  ),
  showlegend=False
)

fig.show()

#### 5. Show Flavourprofile

In [185]:
final_OSA.sum()*10

peaty            0.551070
feinty           4.825058
cereal           2.786105
Green/grassy     6.807675
Floral           8.974375
Fruity           1.112585
Solvently        1.666697
Soapy            6.670317
Sweet           36.902148
Woody           25.442252
Nutty            2.686850
Spicy           23.962436
Oily             1.616430
Sour             3.228415
Sulphury         2.150924
Stale            7.628535
dtype: float64

In [186]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatterpolar(
  r=(np.array(final_OSA.sum()*10)),
  theta=list(final_OSA.columns),
  fill='toself'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
       range=[0, 10]
    ),
  ),
  showlegend=False
)

fig.show()

In [187]:
final_OSA.columns

Index(['peaty', 'feinty', 'cereal', 'Green/grassy', 'Floral', 'Fruity ',
       'Solvently', 'Soapy', 'Sweet', 'Woody', 'Nutty', 'Spicy', 'Oily',
       'Sour', 'Sulphury', 'Stale'],
      dtype='object')

In [188]:
# filter out our 8 comparable OSAs 
comparable_OSAs = ["floral","fruit","wood","complexity","malt","sweet","peat","dried fruits"]
filtered_OSAs = pd.DataFrame(columns=comparable_OSAs)
threshold = 80
# Populate the new DataFrame based on original DataFrame
for column in final_OSA.columns:
    for comparable_OSA in comparable_OSAs:
        if fuzz.ratio(str(column).lower(), comparable_OSA) >= threshold:
            filtered_OSAs[comparable_OSA] = final_OSA[column]

#filtered_OSAs



In [189]:
import plotly.graph_objects as go

panelist_scores = [40,30,50,40,30,40,30]
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[value / 10 for value in panelist_scores],
      theta=comparable_OSAs,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=list(filtered_OSAs.sum()*10),
      theta=comparable_OSAs,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 10]
    )),
  showlegend=False
)

fig.show()

In [190]:
#only compare only columns from final_OSA
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=[0,4,3,4,5,0],
    theta=list(final_OSA.columns),
    fill='toself',
    name="panelist"
))

fig.add_trace(go.Scatterpolar(
  r=(np.array(final_OSA.sum()*10)),
  theta=list(final_OSA.columns),
  fill='toself',
  name= "MS_prediction"))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 10]
    )),
  showlegend=False
)

fig.show()

In [204]:
import plotly.graph_objects as go

#only compare only columns from final_OSA
def compare_profiles(final_OSA, panelists):
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=panelists,
        theta=list(final_OSA.index),
        fill='toself',
        name="panelist"
    ))

    fig.add_trace(go.Scatterpolar(
    r=(np.array(final_OSA)),
    theta=list(final_OSA.index),
    fill='toself',
    name= "MS_prediction"))

    fig.update_layout(
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, 10]
        )),
    showlegend=False
    )

    fig.show()

In [207]:
# look at panelist 1
panelist_data_df = pd.read_table("data/vlb_data_combined.txt", delimiter=',', index_col = 1)
panelist_data_df = panelist_data_df[panelist_data_df["panelist"]==1]
panelist_data_df = panelist_data_df.drop(columns=["panelist", "date"])

In [201]:
overlap_OSAs = ["floral", "fruit", "sweet", "wood", "peat"]

In [202]:
def get_panelist_profile(sample_key):
    panelist_profile=[]
    for OSA in overlap_OSAs :
        panelist_profile.append(int(panelist_data_df.loc[sample_key][OSA]/10))
    return panelist_profile

In [203]:
def find_overlapping_profiles(final_OSA,comparable_OSAs):
    # show only overlap meaning: floral, fruit, sweet, wood, peat
    new_series = pd.Series() 
    threshold = 80
    for comparable_OSA in comparable_OSAs:
        for index in final_OSA.index:
            if fuzz.ratio(str(index).lower(), comparable_OSA) >= threshold:
                new_series[comparable_OSA] = final_OSA[index]
                break
            else:
                new_series[comparable_OSA] = 0
    return(new_series)


In [205]:
def compare_visual(OSA_plotting,sample_key):
    OSA_compare = find_overlapping_profiles(OSA_plotting, overlap_OSAs)
    panelist_compare = get_panelist_profile(sample_key)
    compare_profiles(OSA_compare.T, panelist_compare)

In [211]:
compare_visual(final_OSA.sum()*10,"000920")

C:\Users\Franz\AppData\Local\Temp\ipykernel_15872\3096962253.py:3: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [216]:
compare_visual(final_OSA.mean(axis=0)*600,"000920")

C:\Users\Franz\AppData\Local\Temp\ipykernel_15872\3096962253.py:3: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [191]:

fig = go.Figure()
fig.add_trace(go.Scatterpolar(
  r=[0,4,3,4,5,0],
  theta=["Green","floral","fruit","sweet","wood","spicy"],
  fill='toself'
))


fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 10]
    ),
  ),
  showlegend=False
)

fig.show()